# ***Important***

**Before starting, make sure to read the [Assignment Instructions](https://courseworks2.columbia.edu/courses/172081/pages/assignment-instructions) page on Courseworks2 to learn the workflow for completing this project.**

<ins> Different from project 1 and 2</ins>, apart from the link to your notebook, you are also required to submit 2 additional files namely the collected data `data.pkl` and your chosen model checkpoint `dynamic.pth`.

## Project Setup

In [ ]:
# DO NOT CHANGE

# After running this cell, the folder 'mecs6616_sp23_project3' will show up in the file explorer on the left (click on the folder icon if it's not open)
# It may take a few seconds to appear
!git clone https://github.com/roamlab/mecs6616_sp23_project4.git

Cloning into 'mecs6616_sp23_project4'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 15 (delta 5), reused 6 (delta 0), pack-reused 0
Unpacking objects: 100% (15/15), 8.63 KiB | 1.44 MiB/s, done.


In [ ]:
# DO NOT CHANGE

# move all needed files into the working directory. This is simply to make accessing files easier
!mv /content/mecs6616_sp23_project4/* /content/

In [ ]:
!pip install ray

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.6/58.6 MB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 63.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.5/468.5 kB 30.5 MB/s eta 0:00:00


# Starter Code Explanation

This project uses the same environment as in project 3 which consists of an n linked robot arm. We provide code for teacher dynamics and the controller. We also provide scripts for collecting data, skeleton classes for implementing student dynamics and finally training and evaluating the model.

Similar to last time we define a Robot class inside `robot.py` which provides the interface for controlling the robot arm i.e it provides you with some functions to set/get the state and set the action for the arm and take a step using the `Robot.advance()` method. The state of the arm is a 2n-dimensional vector : n joint positions [rad] + n joint velocities [rad/s] and the action is defined as the n torques (in N-m) applied to n joints respectively.

In `arm_dynamic_base.py` we define the base class for forward dynamics for the arm. The ground truth forward dynamics are defined in `arm_dynamics_teacher.py`. The student dynamics which internally uses a neural network model is defined in `arm_dynamics_student.py`

You are welcome to look in-depth to understand how the ground truth forward dynamics is computed for an arm, given its number of links, link mass, and viscous friction of the environment - however this is not necessary to successfully complete this assignment.


The `models.py` file provides the base class for neural network to learn forward model and also contains skeleton code for you to implement the network architecture for arms with different number of links. We will revisit this again in when providing instructions for training below.

## Part 1: Implement Model Predictive Control

You will implement this controller by completing the MPC class. Specifically, you will implement the compute_action() method by following the algorithm discussed in the lecture. As with previous projects you are free to implement additional methods as needed or change the initialization if need be. While scoring your controller, you will be creating an instance of the MPC class and passing it to the scoring function so ensure that the arguments to the compute_action method remain the same.

Although you do not need to understand how the ArmDynamicsTeacher class works, you could use the compute_fk() method from the class. This will allow you to convert from the state value (represented as array of shape (2*n, 1) where n is num_links) to final end effector position (x, y position of the end effector). Similary we can also compute the velocity of the end effector with the code below:
```
pos_ee = dynamics.compute_fk(state)
vel_ee = dynamics.compute_vel_ee(state)
```


In [ ]:
import numpy as np
from scipy.optimize import minimize

class MPC:
    def __init__(self):
        self.control_horizon = 10
        self.N = 40

    def compute_action(self, dynamics, state, goal, action):
      a = 1.2
      b = 0.014
      c = 0.11
      u = 0.01
      actions = np.broadcast_to(action, (dynamics.get_action_dim(), self.N))
      orig = state
      curr = state
      cost = 0
      pos_ee = np.linalg.norm(dynamics.compute_fk(state))
      for i in range(self.N):
        new_state = dynamics.dynamics_step(curr, actions[:, i], dynamics.dt)
        #new_state = new_state.detach().cpu().numpy().reshape((len(state),1))
        pos_ee = dynamics.compute_fk(new_state)
        vel_ee = dynamics.compute_vel_ee(new_state)
        cost += a*(np.linalg.norm(goal - pos_ee)) + b*np.sum(actions[:, i]) + c*(np.linalg.norm(vel_ee)/(np.linalg.norm(goal - pos_ee)+0.7))
        curr = new_state
      cost = np.absolute(cost)
      curr = orig

      for i in range(20):
        best_cost = np.inf
        best_actions = actions
        for j in range(dynamics.get_num_links()):
          curr_up = orig
          curr_down = orig
          cost_up = 0
          cost_down = 0
          actions_up = np.copy(actions)
          actions_down = np.copy(actions)
          link_up = np.copy(actions[j, :]) + u
          link_down = np.copy(actions[j, :]) - u
          actions_up[j, :] = link_up
          actions_down[j, :] = link_down
          for k in range(self.N):
            new_up = dynamics.dynamics_step(curr_up, actions_up[:, k], dynamics.dt)
            #new_state_up = new_state_up.detach().cpu().numpy().reshape((len(original_state),1))
            pos_up = dynamics.compute_fk(new_up)
            new_down = dynamics.dynamics_step(curr_down, actions_down[:, k], dynamics.dt)
            #new_state_down = new_state_down.detach().cpu().numpy().reshape((len(original_state),1))
            pos_down = dynamics.compute_fk(new_down)
            vel_up = dynamics.compute_vel_ee(new_up)
            vel_down = dynamics.compute_vel_ee(new_down)
            cost_up += a*(np.linalg.norm(goal - pos_up)) + b*np.sum(actions_up[:, k]) + c*(np.linalg.norm(vel_up)/(np.linalg.norm(goal - pos_up)+0.7))
            cost_down += a*(np.linalg.norm(goal - pos_down)) + b*np.sum(actions_down[:, k]) + c*(np.linalg.norm(vel_down)/(np.linalg.norm(goal - pos_down)+0.7))
            curr_up = new_up
            curr_down = new_down

          cost_up = np.absolute(cost_up)
          cost_down = np.absolute(cost_down)

          if cost_up < cost_down:
            if cost_up < best_cost:
              best_cost = cost_up
              best_actions = actions_up
          elif cost_up > cost_down:
            if cost_down < best_cost:
              best_cost = cost_down
              best_actions = actions_down
        if best_cost < cost:
          cost = best_cost
          actions = best_actions

      return np.reshape(actions[:, 0], (dynamics.get_num_links(), 1))

# Manually testing the controller
This part is for you to manually check the performance of your controller before you are ready for it be evaluated by our scoring function.
To test your implementation run the following code. Feel free to play around with the cell or change the num_links / goal positions . You can define your controller however you would like to and then switch on gui to see how close your end effectors get to the goal position

Every time step within the environment is 0.01s, which is defined in the dynamics as `dt`.

The MPC class has a `control_horizon` variable which represents the frequency at which `controller.compute_action()` will be called

In the scoring function you will be evaluated on the distance of your end effector to the goal position and the velocity of the end effector.

In [ ]:
import sys
import numpy as np
from arm_dynamics_teacher import ArmDynamicsTeacher
from robot import Robot
from render import Renderer
from score import *
import torch
import time
import math
np.set_printoptions(suppress=True)

# Teacher arm with 3 links
dynamics_teacher = ArmDynamicsTeacher(
    num_links=3,
    link_mass=0.1,
    link_length=1,
    joint_viscous_friction=0.1,
    dt=0.01)

arm = Robot(dynamics_teacher)
arm.reset()

gui = False

if gui:
  renderer = Renderer()
  time.sleep(1)

# Controller
controller = MPC()

# Resetting the arm will set its state so that it is in the vertical position,
# and set the action to be zeros
arm.reset()

# Choose the goal position you would like to see the performance of your controller
goal = np.zeros((2, 1))
goal[0, 0] = 2.5
goal[1, 0] = -0.7
arm.goal = goal

dt = 0.01
time_limit = 5
num_steps = round(time_limit/dt)

# Control loop
for s in range(num_steps):
  t = time.time()
  arm.advance()

  if gui:
    renderer.plot([(arm, "tab:blue")])
  time.sleep(max(0, dt - (time.time() - t)))

  if s % controller.control_horizon==0:
    state = arm.get_state()

    # Measuring distance and velocity of end effector
    pos_ee = dynamics_teacher.compute_fk(state)
    dist = np.linalg.norm(goal-pos_ee)
    vel_ee = np.linalg.norm(arm.dynamics.compute_vel_ee(state))
    print(f'At timestep {s}: Distance to goal: {dist}, Velocity of end effector: {vel_ee}')

    action = arm.action
    action = controller.compute_action(arm.dynamics, state, goal, action)
    arm.set_action(action)
    #print(f'{s}: state: {state.reshape((2*dynamics_teacher.num_links,1))}, action: {action.reshape((dynamics_teacher.num_links,1))}')

At timestep 0: Distance to goal: 3.3970575502926055, Velocity of end effector: 5.539171676143414e-18
At timestep 10: Distance to goal: 3.3700694398297704, Velocity of end effector: 0.4711885209845729
At timestep 20: Distance to goal: 3.2917972065334293, Velocity of end effector: 0.8281741754765136
At timestep 30: Distance to goal: 3.1525007932809657, Velocity of end effector: 1.0354721566284708
At timestep 40: Distance to goal: 2.947066456960029, Velocity of end effector: 1.11947025322798
At timestep 50: Distance to goal: 2.6834354436087184, Velocity of end effector: 1.0485247276079983
At timestep 60: Distance to goal: 2.4395169554959923, Velocity of end effector: 0.6550021782857365
At timestep 70: Distance to goal: 2.29873906214577, Velocity of end effector: 0.29989022638915597
At timestep 80: Distance to goal: 2.267473038773757, Velocity of end effector: 0.13787567268440903
At timestep 90: Distance to goal: 2.2917608784042995, Velocity of end effector: 0.2123406883827209
At timestep 

## Grading and Evaluation for Part 1
Your controller will be graded on 6 tests. 2 tests each for the 1-link ,2-link, and 3-link arms. The arm will start off in the initial state with the arms pointing stright down. The testing criteria depend on the distance and the velocity of the end effectors . Each test will run the robot arm for **5.0 seconds**. At the end of the 5 seconds the test will be:

A success if your end effectors meet this criteria:
`distance_to_goal < 0.1 and vel_ee < 0.5`

A partial success if your end effectors meet this criteria:
`distance_to_goal < 0.2 and vel_ee < 0.5`


In [ ]:
# Scoring using score_mpc
controller = MPC()
gui = False

In [ ]:
# DO NOT CHANGE
score_mpc_true_dynamics(controller, gui)


Part1: EVALUATING CONTROLLER (with perfect dynamics)
-----------------------------------------------------
NUM_LINKS: 1
Test  1
Success! :)
 Goal: [ 0.38941834 -0.92106099], Final position: [ 0.38776116 -0.92175988], Final velocity: [0.]
score: 1.5/1.5
Test  2
Success! :)
 Goal: [-0.68163876 -0.73168887], Final position: [-0.68358302 -0.72987276], Final velocity: [0.]
score: 1.0/1.0
NUM_LINKS: 2
Test  1
Success! :)
 Goal: [ 0.6814821  -1.61185674], Final position: [ 0.67891771 -1.61451023], Final velocity: [0.00099123]
score: 1.5/1.5
Test  2
Success! :)
 Goal: [-1.19286783 -1.28045552], Final position: [-1.19033105 -1.28403032], Final velocity: [0.05379889]
score: 1.0/1.0
NUM_LINKS: 3
Test  1
Success! :)
 Goal: [ 1.29444895 -2.36947292], Final position: [ 1.29428909 -2.36853645], Final velocity: [0.00882273]
score: 1.5/1.5
Test  2
Success! :)
 Goal: [-2.10367746 -1.35075576], Final position: [-2.05807184 -1.37593074], Final velocity: [0.0266228]
score: 1.0/1.0
       
----------------

### Part 2.1: Model Architecture
Beyond this point you will be focusing on the 2 link arm only.

We have a base class Model and a subclass for the 2-link arm. The class Model is a base class for our models.  In compute_next_state() method,
you have to use the trick to use joint accelerations to compute the next state similar to what you did in Project 3.

In the `Model2Link` class you will use a neural network to compute the joint accelerations by implementing `compute_qddot()` method. This will take 6 values (2 joint angles, 2 joint velocities and 2 actions applied to the arm) and output 2 joint acceleration values

Do not change the arguments for the `__init__()` method even if you do not use them.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

class Model(nn.Module):
	def __init__(self, num_links, time_step):
		super().__init__()
		self.num_links = num_links
		self.time_step = time_step

	def forward(self, x):
		qddot = self.compute_qddot(x)
		state = x[:, :2*self.num_links]
		next_state = self.compute_next_state(state, qddot)
		return next_state

	def compute_next_state(self, state, qddot):
		# Your code goes here
		next_state = np.zeros_like(state)
		next_state1 = torch.from_numpy(next_state)
		state1 = state
		qddot1 = qddot
		next_state1[:,self.num_links: 2*self.num_links] = state1[:,self.num_links: 2*self.num_links] + qddot1*self.time_step
		next_state1[:,0:self.num_links] = state1[:,0:self.num_links] + state1[:,self.num_links: 2*self.num_links]*self.time_step + 0.5*qddot1*(self.time_step*self.time_step)
		#torch.reshape(next_state1, (4,1))
		return next_state1

	def compute_qddot(self, x):
		pass

class Model2Link(Model):
	def __init__(self, time_step):
		super().__init__(2, time_step)
		self.fc1 = nn.Linear(6, 64)
		self.fc2 = nn.Linear(64, 64)
		self.fc3 = nn.Linear(64, 2)
		self.relu = nn.ReLU()
    # Your code goes here

	def compute_qddot(self, x):
    # Your code goes here
			x = self.fc1(x)
			x = self.relu(x)
			x = self.fc2(x)
			x = self.relu(x)
			x = self.fc3(x)
			return x


### Part 2.2: Collect Data
Similar to project 3, we will collect data which will be used to learn a forward model for our 2 link robot arm. Once we have learnt a forward model you will be evaluated on your MPC Controller that uses the learnt dynamics model instead of the true dynamics.

You can modify the collect_data function or write any of your own functions however you choose to. You will be evaluated on the **2 Link Robot**

In [ ]:
from IPython.core.display import json
# Teacher arm with 3 links
dynamics_teacher = ArmDynamicsTeacher(
    num_links=2,
    link_mass=0.1,
    link_length=1,
    joint_viscous_friction=0.1,
    dt=0.01)

arm = Robot(dynamics_teacher)
arm.reset()

def collect_data(arm):

 # ---
  # You code goes here. Replace the X, and Y by your collected data
  # Control the arm to collect a dataset for training the forward dynamics.
  '''
  gui = False

  if gui:
    renderer = Renderer()
  time.sleep(1)

  # Controller
  controller = MPC()

  # Resetting the arm will set its state so that it is in the vertical position,
  # and set the action to be zeros
  arm.reset()

  # Choose the goal position you would like to see the performance of your controller

  dt = 0.01
  time_limit = 2.5
  num_steps = round(time_limit/dt)

  num_trials = 10
  points = np.zeros((2,num_trials))
  xval = np.linspace(0.05,1.95,num = num_trials)
  yval =
  points[0,:] = xval
  points[1,:] = yval

  X = np.zeros((num_steps*num_trials, arm.dynamics.get_state_dim() + arm.dynamics.get_action_dim()))
  Y = np.zeros((num_steps*num_trials, arm.dynamics.get_state_dim()))

  for j in range(num_trials):

    goal = np.zeros((2, 1))
    goal[0, 0] = points[0,j]
    goal[1, 0] = points[1,j]
    arm.goal = goal
    print(goal)

    for s in range(num_steps):
      t = time.time()
      arm.advance()

      if gui:
        renderer.plot([(arm, "tab:blue")])
      time.sleep(max(0, dt - (time.time() - t)))

      if s % controller.control_horizon==0:
        state = arm.get_state()

        # Measuring distance and velocity of end effector
        pos_ee = dynamics_teacher.compute_fk(state)
        dist = np.linalg.norm(goal-pos_ee)
        vel_ee = np.linalg.norm(arm.dynamics.compute_vel_ee(state))
        print(f'At timestep {s}: Distance to goal: {dist}, Velocity of end effector: {vel_ee}')

        action = arm.action
        action = controller.compute_action(arm.dynamics, state, goal, action)
        arm.set_action(action)

        next_state = arm.dynamics.dynamics_step(state,action,0.01)
        k = j*num_steps + s
        X[k, :] = np.concatenate((state.flatten(), action.flatten()))
        Y[k, :] = next_state.flatten()
      #print(f'At timestep {i}: Distance to goal: {dist}, Velocity of end effector: {vel_ee}')
      #print(f"step: {i}, X: {X[i,:]}, Y: {Y[i,:]}")
      #print(f'step {i}')
  '''

  gui = False

  if gui:
    renderer = Renderer()
  time.sleep(1)

  # Controller
  controller = MPC()

  # Resetting the arm will set its state so that it is in the vertical position,
  # and set the action to be zeros
  arm.reset()

  # Choose the goal position you would like to see the performance of your controller
  goal = np.zeros((2, 1))
  goal[0, 0] = 1.3
  goal[1, 0] = -0.7
  arm.goal = goal

  dt = 0.01
  time_limit = 10
  num_steps = round(time_limit/dt)

  X = np.zeros((num_steps, arm.dynamics.get_state_dim() + arm.dynamics.get_action_dim()))
  Y = np.zeros((num_steps, arm.dynamics.get_state_dim()))
  state = np.zeros((4,1))

  # Control loop
  for i in range(num_steps):
    t = time.time()
    arm.advance()

    if gui:
      renderer.plot([(arm, "tab:blue")])
    time.sleep(max(0, dt - (time.time() - t)))

    if i % controller.control_horizon==0:
      state = arm.get_state()

      # Measuring distance and velocity of end effector
      pos_ee = dynamics_teacher.compute_fk(state)
      dist = np.linalg.norm(goal-pos_ee)
      vel_ee = np.linalg.norm(arm.dynamics.compute_vel_ee(state))
      print(f'At timestep {i}: Distance to goal: {dist}, Velocity of end effector: {vel_ee}')

      action = arm.action
      action = controller.compute_action(arm.dynamics, state, goal, action)
      arm.set_action(action)
      next_state = arm.dynamics.advance(state,action)
      X[i, :] = np.concatenate((state.flatten(), action.flatten()))
      Y[i, :] = next_state.flatten()
      #print(f"step: {i}, X: {X[i,:]}, Y: {Y[i,:]}")


  # ---

  return X, Y


In [ ]:
import pickle

# Call the function you have defined above to collect data
X, Y = collect_data(arm)
save_dir = 'dataset'
if not os.path.exists(save_dir):
  os.makedirs(save_dir)

# Save the collected data in the data.pkl file
data = {'X': X, 'Y': Y}
pickle.dump(data, open(os.path.join(save_dir, 'data.pkl'), "wb" ))

At timestep 0: Distance to goal: 1.8384776310850235, Velocity of end effector: 7.715274834628325e-18
At timestep 10: Distance to goal: 1.811591747078342, Velocity of end effector: 0.5176842655249215
At timestep 20: Distance to goal: 1.7278930644944233, Velocity of end effector: 1.007222878116306
At timestep 30: Distance to goal: 1.578859456883025, Velocity of end effector: 1.2321523186649694
At timestep 40: Distance to goal: 1.3757853181078932, Velocity of end effector: 1.2785670905159776
At timestep 50: Distance to goal: 1.1633500716453227, Velocity of end effector: 0.9305124928093426
At timestep 60: Distance to goal: 0.9960983287667095, Velocity of end effector: 0.44022879617422933
At timestep 70: Distance to goal: 0.8930275306380792, Velocity of end effector: 0.0745485646324283
At timestep 80: Distance to goal: 0.8203843912969085, Velocity of end effector: 0.630135848327024
At timestep 90: Distance to goal: 0.7319246624281275, Velocity of end effector: 1.2201930135838297
At timestep

### Part 2.3: Training the forward model
By now you would be familiar with the basic skeleton of training a forward model.

The starter code already creates the dataset class and loads the dataset with a random 0.8/0.2 train/test split for you. This script should save the model that it trains. You should use a specific procedure for saving, outlined below.

In machine learning, it is a very good practice to save not only the final model but also the checkpoints. Our starter code already configures save_dir for you and for each epoch of your training, you should use the following code to save the model at that epoch.

```
model_folder_name = f'epoch_{epoch:04d}_loss_{test_loss:.8f}'
if not os.path.exists(os.path.join(args.save_dir, model_folder_name)):
    os.makedirs(os.path.join(args.save_dir, model_folder_name))
torch.save(model.state_dict(), os.path.join(args.save_dir, model_folder_name, 'dynamics.pth'))
print(f'model saved to {os.path.join(args.save_dir, model_folder_name, "dynamics.pth")}\n')
```
The output from running this code should be a folder as below:

```
models/
    2021-03-24_23-57-50/
        epoch_0001_loss_0.00032930/
            dynamics.pth
        epoch_0002_loss_0.00009413/
            dynamics.pth   
        ...  
```
You can implement the functions below as you please to collect data

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import tqdm
import pickle
import torch.optim as optim
import argparse
import time

class DynamicDataset(Dataset):
  def __init__(self, datafile):
    data = pickle.load(open(datafile, 'rb'))
    # X: (N, 6), Y: (N, 4)
    self.X = data['X'].astype(np.float32)
    self.Y = data['Y'].astype(np.float32)


  def __len__(self):
    return self.X.shape[0]

  def __getitem__(self, idx):
    return self.X[idx], self.Y[idx]


def train_one_epoch(model, train_loader, optimizer, criterion):
  model.train()
	# ---
	# Your code goes here
  train_loss = 0.0
  for batch_idx, (data, target) in enumerate(train_loader):
      optimizer.zero_grad()
      output = model(data)
      loss = criterion(output, target)
      loss.backward()
      optimizer.step()
      train_loss += loss.item()
  return train_loss / len(train_loader)

	# ---


def test(model, test_loader, criterion):
  model.eval()
	# --
	# Your code goes here
  test_loss = 0.0
  with torch.no_grad():
    for batch_idx, (data, target) in enumerate(test_loader):
      #data, target = data.to(device), target.to(device)
      output = model(data)
      loss = criterion(output, target)
      test_loss += loss.item()
  return test_loss / len(test_loader)
  # --

def train_forward_model():

	# --
	# Implement this function
  # --

  # Keep track of the checkpoint with the smallest test loss and save in model_path
  model_path = None
  max_test_loss = 1e4
  model = Model2Link(0.01)

  datafile = 'dataset/data.pkl'
  split = 0.2
  dataset = DynamicDataset(datafile)
  dataset_size = len(dataset)
  test_size = int(np.floor(split * dataset_size))
  train_size = dataset_size - test_size
  train_set, test_set = random_split(dataset, [train_size, test_size])

  train_loader = torch.utils.data.DataLoader(train_set, batch_size=100, shuffle=True)
  test_loader = torch.utils.data.DataLoader(test_set, batch_size=100, shuffle=True)

  # The name of the directory to save all the checkpoints
  timestr = time.strftime("%Y-%m-%d_%H-%M-%S")
  model_dir = os.path.join('models', timestr)

  optimizer = optim.Adam(model.parameters(), lr=0.001)
  criterion = nn.MSELoss()

  epochs=100
  for epoch in range(1, 1 + epochs):
    # --
    # Your code goes here
    # Train one epoch
        train_loss = train_one_epoch(model, train_loader, optimizer, criterion)

        # Test the model
        test_loss = test(model, test_loader, criterion)

        folder = f'epoch_{epoch:04d}_loss_{test_loss:.8f}'
        if not os.path.exists(os.path.join(model_dir, folder)):
          os.makedirs(os.path.join(model_dir, folder))
        torch.save(model.state_dict(), os.path.join(model_dir, folder, 'dynamics.pth'))

        if model_path is None or test_loss < max_test_loss:
          max_test_loss = test_loss
          model_path = os.path.join(model_dir, folder, 'dynamics.pth')
          print(model_path)

        print(f'Epoch [{epoch}/{epochs}] Train Loss: {train_loss:.11f} Test Loss: {test_loss:.11f}')
    # --


  return model_path

In [ ]:
model_path = train_forward_model()

models/2023-04-16_03-52-07/epoch_0001_loss_0.00001968/dynamics.pth
Epoch [1/100] Train Loss: 0.00010953208 Test Loss: 0.00001968396
models/2023-04-16_03-52-07/epoch_0002_loss_0.00001948/dynamics.pth
Epoch [2/100] Train Loss: 0.00010843930 Test Loss: 0.00001947767
models/2023-04-16_03-52-07/epoch_0003_loss_0.00001941/dynamics.pth
Epoch [3/100] Train Loss: 0.00010795691 Test Loss: 0.00001941152
models/2023-04-16_03-52-07/epoch_0004_loss_0.00001938/dynamics.pth
Epoch [4/100] Train Loss: 0.00010769417 Test Loss: 0.00001938397
Epoch [5/100] Train Loss: 0.00010740867 Test Loss: 0.00001938963
models/2023-04-16_03-52-07/epoch_0006_loss_0.00001919/dynamics.pth
Epoch [6/100] Train Loss: 0.00010683773 Test Loss: 0.00001919491
models/2023-04-16_03-52-07/epoch_0007_loss_0.00001914/dynamics.pth
Epoch [7/100] Train Loss: 0.00010642738 Test Loss: 0.00001914479
models/2023-04-16_03-52-07/epoch_0008_loss_0.00001910/dynamics.pth
Epoch [8/100] Train Loss: 0.00010594879 Test Loss: 0.00001910398
models/2023

### Part 2.4: Completing ArmDynamicsStudent

After you are done with training, you need to complete ArmDynamicsStudent class following the comments below to load the saved checkpoint (in function init_model) and then use it to predict the new state given the current state and action (in function dynamics_step). Please do not modify the arguments to those functions, even though you might not use all of them.

In [ ]:
from arm_dynamics_base import ArmDynamicsBase

class ArmDynamicsStudent(ArmDynamicsBase):
    def init_model(self, model_path, num_links, time_step, device):
        # ---
        # Your code hoes here
        # Initialize the model loading the saved model from provided model_path
        self.model = Model2Link(time_step)
        self.model.load_state_dict(torch.load(model_path))
        # ---
        self.model_loaded = True

    def dynamics_step(self, state, action, dt):
        if self.model_loaded:
            # ---
            # Your code goes here
            # Use the loaded model to predict new state given the current state and action
          with torch.no_grad():
                state_tensor = torch.tensor(state, dtype=torch.float32).view(1,-1)
                action_tensor = torch.tensor(action, dtype=torch.float32).view(1,-1)
                inputs = torch.cat((state_tensor, action_tensor), dim=1)
                torch.reshape(inputs, (6,1))
                op = self.model(inputs)
                op = op.cpu().detach().numpy().reshape((4,1))
                new_state = op
          return new_state
            # ---
        else:
            return state

### Manually Testing the MPC Controller with the learnt dynamics model
We will now use the learnt dynamics model that you have trained. The model is loaded in the dynamics.init_model method. You can modify the goal positions to see how well is the controller performing similar to what you did before. Feel free to play around with the code in this cell to test your performance before the grading part.


In [ ]:
import sys
import numpy as np
from arm_dynamics_teacher import ArmDynamicsTeacher
from robot import Robot
from render import Renderer
from score import *
import torch
import time

# Teacher arm with 3 links
dynamics_teacher = ArmDynamicsTeacher(
    num_links=2,
    link_mass=0.1,
    link_length=1,
    joint_viscous_friction=0.1,
    dt=0.01)

arm = Robot(dynamics_teacher)
arm.reset()

gui = False
action = np.zeros((arm.dynamics.get_action_dim(), 1))
if gui:
  renderer = Renderer()
  time.sleep(1)


# Controller
controller = MPC()
dynamics_student = ArmDynamicsStudent(
    num_links=2,
    link_mass=0.1,
    link_length=1,
    joint_viscous_friction=0.1,
    dt=0.01)
device = torch.device('cpu')

# model_path should have the path to the best model that you have trained so far
# which you would like to use for testing the controller
model_path = "models/2023-04-16_03-52-07/epoch_0042_loss_0.00000035/dynamics.pth"
dynamics_student.init_model(model_path, 2, 0.01, device)

# Control loop
action = np.zeros((arm.dynamics.get_action_dim(), 1))
goal = np.zeros((2, 1))
goal[0, 0] = -1.3
goal[1, 0] = 0.5
arm.goal = goal

dt = 0.01
time_limit = 2.5
num_steps = round(time_limit/dt)
for s in range(num_steps):
  t = time.time()
  arm.advance()

  if gui:
    renderer.plot([(arm, "tab:blue")])
  time.sleep(max(0, dt - (time.time() - t)))

  if s % controller.control_horizon==0:
    state = arm.get_state()
    pos_ee = dynamics_teacher.compute_fk(state)
    dist = np.linalg.norm(goal-pos_ee)
    vel_ee = np.linalg.norm(arm.dynamics.compute_vel_ee(state))
    print(f'At timestep {s}: Distance to goal: {dist}, Velocity of end effector: {vel_ee}')
    action = controller.compute_action(dynamics_student, state, goal, action)
    arm.set_action(action)


At timestep 0: Distance to goal: 2.817800560721074, Velocity of end effector: 7.715274834628325e-18
At timestep 10: Distance to goal: 2.8192465714941113, Velocity of end effector: 0.23185132272663753
At timestep 20: Distance to goal: 2.81609374184526, Velocity of end effector: 0.5297410652897245
At timestep 30: Distance to goal: 2.7958825045446374, Velocity of end effector: 0.820048788453333
At timestep 40: Distance to goal: 2.748151327776294, Velocity of end effector: 1.084574782159061
At timestep 50: Distance to goal: 2.667338847646074, Velocity of end effector: 1.3273105320157912
At timestep 60: Distance to goal: 2.5511427704362157, Velocity of end effector: 1.5516030708995772
At timestep 70: Distance to goal: 2.3984437779618024, Velocity of end effector: 1.757988059244895
At timestep 80: Distance to goal: 2.212113552487022, Velocity of end effector: 1.7594888740230457
At timestep 90: Distance to goal: 2.0030289394822747, Velocity of end effector: 1.2906368041840124
At timestep 100:

## Grading and Evaluation of Part 2
You will be evaluated on how well your controller+learnt dynamics works together. The scoring functions consists of 16 random test goals all of which will be below the x axis and between 0.05 to 1.95 lengths away from the origin.
The controller will call the compute_action method from your MPC class and apply the action for 10 timesteps
```
action = controller.compute_action(dynamics_student, state, goal, action)
```

Each test will run the robot arm for **2.5 seconds**. At the end of the 2.5 seconds the test will be:

A success if your end effectors meet this criteria:
`distance_to_goal < 0.2 and vel_ee < 0.5`

A partial success if your end effectors meet this criteria:
`distance_to_goal < 0.3 and vel_ee < 0.5`

You need 15 out of the 16 tests to succeed to get a full score

In [ ]:
controller = MPC()
dynamics_student = ArmDynamicsStudent(
    num_links=2,
    link_mass=0.1,
    link_length=1,
    joint_viscous_friction=0.1,
    dt=0.01)
model_path = 'models/2023-04-16_03-52-07/epoch_0042_loss_0.00000035/dynamics.pth'
gui=False

In [ ]:
# DO NOT CHANGE
score_mpc_learnt_dynamics(controller, dynamics_student, model_path, gui)

Part2: EVALUATING CONTROLLER + LEARNED DYNAMICS
-----------------------------------------------
NUM_LINKS: 2
Test  1
Fail :(
 Goal: [-0.83931506 -0.17109352], Final position: [-0.59318193 -1.33531289], Final velocity: [0.08169862]
score: 0/0.5
Test  2
Fail :(
 Goal: [ 0.27206229 -0.1323684 ], Final position: [-0.8017003  0.0386893], Final velocity: [0.62254448]
score: 0/0.5
Test  3
Fail :(
 Goal: [-0.53098054 -0.98812195], Final position: [-0.08675118  0.07278706], Final velocity: [0.33553007]
score: 0/0.5
Test  4
Fail :(
 Goal: [ 0.20637967 -0.28259279], Final position: [-1.10665201 -0.85855824], Final velocity: [0.19507219]
score: 0/0.5
Test  5


KeyboardInterrupt: ignored

Hints and Suggestions:
1. You can use your MPC Controller in your data collection to gather better training samples
2. A good cost function to evaluate your trajectory in MPC is very important and you can use both distance and velocity metrics to define the cost function.
3. As mentioned in the lecture, a constant torque with pseudo gradients seems to work well for this project. You can also use multiple delta values to gather more trajectories to choose from.
4. Since we are passing the MPC object to the controller you can instantiate the MPC class with different parameters like the planning horizon, delta values etc.
5. To speed up data collection, avoid using np.concatenate(), np.stack() or np.append() like functions on your X and Y arrays. Instead, initialize X and Y arrays with all zeros using the correct shape and then fill in the values one by one. This is much faster in numpy